importiamo emnist

In [ ]:
!pip install git+https://github.com/hosford42/EMNIST.git

Ecco di seguito il box con gli import e il mount del drive:

In [ ]:
"""ecco gli import"""
from google.colab import files
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from emnist import list_datasets, extract_training_samples
import matplotlib.pyplot as plt
from random import randint
import random as rand
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')

definiamo il modello del mondello XD

In [ ]:
from google.colab import files
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from emnist import list_datasets, extract_training_samples
import matplotlib.pyplot as plt
from random import randint
import random as rand

#caricamento del dataset di emnist (bymerge)
emnist_images, emnist_labels = extract_training_samples('bymerge')

#filtraggio del dataset
bgty = [11, 16, 29, 34]
letters_cost = {'11': 1, '16':3, '29':0, '34':2}

#ora filtriamo il dataset scegliendo solo le lettere che ci interessano: B, G, T, Y
filtred_indices = [i for i, label in enumerate(emnist_labels) if label in bgty]
filtred_images = [emnist_images[i] for i in filtred_indices]
filtred_labels = [letters_cost[str(emnist_labels[i])] for i in filtred_indices]

#normalizzazione delle immagini
filtred_images = np.array(filtred_images).astype('float32') / 255
filtred_labels = np.array(filtred_labels)

#dividiamo il dataset in training e validation set:
X_train, X_val, y_train, y_val = train_test_split(filtred_images, filtred_labels, test_size=0.3, random_state = 100)

#Visualizzazione di campione di immagini del training set
t = randint(0, len(X_train) - 25)
plt.figure(figsize = (10, 10))
for i, image in enumerate(X_train[t:t+25]):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
plt.show()

#definiamo il modello di rete neurale:
vector = 784    #perchè le immagini di emnist sono di dimensioni 28x28 = 784
X_train = X_train.reshape(X_train.shape[0], vector)
X_val = X_val.reshape(X_val.shape[0], vector)

filtred_model = keras.Sequential([
    layers.Dropout(0.5, input_shape=(vector,)),
    layers.Dense(1024, kernel_initializer="lecun_normal", activation = "selu"),
    layers.Dense(2048, activation="relu"),
    layers.Dense(1024, activation="relu"),
    layers.Dense(4, activation="softmax")
])

filtred_model.summary()

#Compilazione e addestramento del modello
filtred_model.compile(loss = "sparse_categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
history = filtred_model.fit(X_train, y_train, batch_size = 256, epochs=10, validation_data=(X_val, y_val))

#valutazione del modello
score = filtred_model.evaluate(X_val, y_val, verbose = 0)
print(f"Test loss: {score[0]}\nTest Accuracy: {score[1]}")

#singola previsione e visualizzazione:
one_case = rand.randrange(0, len(X_val))
prediction_single = filtred_model.predict(X_val[one_case:one_case + 1])
print(prediction_single)            #stampa le probabilità su ogni lettera
print(np.sum(prediction_single))    #dovrebbe tornare 1 perchè è la somma delle probabilità


#questo codice trova la classe con probabilità più alta (quale è la lettera più probabile che sia)
y_classes = prediction_single.argmax()      #argmax() retituisce la classe con prob max
print(f"la predizione è: {y_classes}")

x_test_vis = X_val[one_case:one_case+1].reshape(28, 28) #modifica il vettore di visualizzazione in matrice 28*28 in maniera da poter essere stampato
plt.imshow(x_test_vis, cmap=plt.cm.binary)
plt.show()

#plot delle metriche di addestramento:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Accuratezza del modello")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(['Train,', 'Val'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss del Modello')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc = 'upper left')
plt.show()

#salvataggio e download del modello
filtred_model.save('modello.keras')
files.download('modello.keras')

#salvataggio nel cloud


#Crea una directory se non esiste:
if not os.path.exists('/content/drive/MyDrive/modelli_keras/'):
    os.makedirs('/content/drive/MyDrive/modelli_keras/')

#Copia il modello sul drive:
import shutil
shutil.copy('modello.keras', '/content/drive/MyDrive/modelli_keras/modello.keras')
print("Modello salvato su Google Drive!")

Cerchiamo di passare al codice il blocchino di info da aima python che abbiamo sul drive condivisong


In [ ]:
import zipfile, sys, os

#monto il drive
drive.mount('/content/drive')

#definisco i percorsi:
zip_path = '/content/drive/MyDrive/aima/aima.zip'
extract_to = '/content/aima' #viene estratto in una cartella temporanea di colab per maggiore velocità di esecuzione e lettura

#verifichiamo che il file esista (lo zip):
if os.path.exists(zip_path):
    #estraiamo
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f'Libreria estratta con successo nel path: {extract_to}')

    #Aggiungiamo la libreria al path
    sys.path.insert(0, extract_to)
    print(f'Libreria Pronta!')
else:
    print(f'Errore File non trovato in {zip_path}')

ora definiamo i comandi / movimenti che il modello può considerare di fare:

In [ ]:
#Qui dobbiamo controllare se serve veramente l'import di search

#importo la libreria di os per interfaccarmi con il sistema della macchina ospite
import os
import sys
aima_path = '/content/aima'
if aima_path not in sys.path:
    sys.path.insert(0, aima_path)
#modifichiamo il puntatore della directory dalla posizione attuale a la nuova per caricare la libreria
os.chdir('/content/aima')
from aima import search
from aima.search import *
#Funzioen che procede a prendere gli indici delle lettere/colori e le mappa
#direttamente al loro peso corrispettivo
def pesoColore(colore) :
  risultato = None
  if colore == '11' : #Lettera B
      risultato = 1
  elif colore == '34' : #Lettera Y
    risultato = 2
  elif colore == '16' : #Lettera G
    risultato = 3
  elif colore == '29' : #Lettera T
    risultato = 0
  else :
    print(f"Non e' stato riconosciuto il colore all'interno della cella")
  return risultato


class ProblemaColori(Problem) :
    #inizializzazione del problema
    def __init__(self, initial, larghezza, altezza) :
        self.l = larghezza
        self.a = altezza
        initial = list(initial)
        #Def posizione della testina
        self.T = len(initial) - 1
        self.posStart = initial[self.T]

        #Def. dei tre possibili stati finali del problema che vorremmo avere
        self.one = [1 for a in range(larghezza*altezza)]
        self.one[self.posStart] = 0 #Testina tornata al suo posto
        self.two = [2 for a in range(larghezza*altezza)]
        self.two[self.posStart] = 0
        self.three = [3 for a in range(larghezza*altezza)]
        self.three[self.posStart] = 0
        #Per Carlo in pratica i tre casi finali sono tre griglie che sono colorate uniformemente
        #tranne per la posizione della testina

        super().__init__(initial, goal = (self.one, self.two, self.three))

    def actions(self, state) :
        #Queste sotto sono le azione possibili che la testina/alg. puo eseguire normalmente non
        #tenendo conto dei casi limiti per ora
        testina = state[self.T]
        movPoss = ['destra', ' sinistra', 'su', 'giu', 'coloraBlu', 'coloraVerde', 'coloraGiallo']

        #Dobbiamo togliere le prime azioni non possibili
        #BORDI DELLA GRIGLIA
        if testina % self.l == self.l - 1 : #Es. Carlo Questo e' quando si trova sul bordo destro della matrice
          movPoss.remove('destra')
        if testina % self.l == 0 :
          movPoss.remove('sinistra') #Caso opposto a quello sopra, siamo tutto a sinistra
        if testina < self.w :
          movPoss.remove('su')
        if testina >= self.w * (self.h - 1) :
          movPoss.remove('giu')

        #Per Carlo: Non facciamo ancora un return della lista dei movimenti perche' viene piu comodo
        #fare tutti i controlli prima e dopo dare una lista svuotata di tutti i movimenti erronei
        #COLORAZIONE
        #Caso nel quale abbiamo finito di colorare tutta la griglia
        if state[0 : len(state) - 1] in self.goal :
          movPoss.remove('coloraBlu')
          movPoss.remove('coloraVerde')
          movPoss.remove('coloraGiallo')

        else :
          #Se ho colorato la cella di un colore e' inutile averlo tra le opzioni
          if state[testina] == 1 :
            movPoss.remove('coloraBlu')
          if state[testina] == 2 :
            movPoss.remove('coloraGiallo')
          if state[testina] == 3 :
            movPoss.remove('coloraVerde')
          if state[testina] == 0 :
            movPoss.remove('coloraBlu')
            movPoss.remove('coloraGiallo')
            movPoss.remove('coloraVerde')

        return movPoss

    def result(self, state, action) :
      #Logica per capire lo stato dopo un'azione
        testina = state[self.T]
        new_state = state[0 : self.T] #Ho dovuto correggere, non avevo messo questo

        if(action == 'coloraBlu' or action == 'coloraGiallo' or action == 'coloraVerde') :
          colora = {'coloraBlu' : 1, 'coloraGiallo' : 2, 'coloraVerde' : 3}
          new_state[testina] = colora[action]
        else :
          mov = {'destra' : 1, 'sinistra' : -1, 'su' : -self.l, 'giu' : self.l}
          testina += mov[action]

        state = new_state + [testina]
        return state

    #Verifichiamo che la matrice (senza la testina) sia uno degli stati goal
    #definiti sopra
    def matriceGoal(self, state) :
        return state[0 : len(state) - 1] in self.goal

    #Ci assicuriamo che la testina (come da traccia) sia tornata alla cella di partenza
    def cellaInit(self, state) :
        return state[self.T] == self.posStart

    #Calcolo del costo di tutti i movimenti
    def costo(self, c, state1, action, state2) :
      if action in ('coloraBlu', 'coloraVerde', 'coloraGiallo') :
            colore = {'coloraBlu' : '2', 'coloraGiallo' : '25', 'coloraVerde' : '7'}[action]
            return c + pesoColore(colore)
      else:
            return c + 1

pre processing: preparazione dei dati
predizione: caricamento del modello e previsione delle probabilità
post processing: determinazinoe delle label più probabili nell'immagine passata.

In [ ]:
import cv2 as cv
from scipy import ndimage
import matplotlib.pyplot as plt
from base64 import b64decode
from tensorflow import keras
from tensorflow.keras.models import load_model

from google.colab import files

#drive.mount('/content/drive')
modello_filtrato = keras.models.load_model('/content/drive/MyDrive/modelli_keras/modello.keras')

def import_image():
    image = files.upload()
    name_image = list(image.keys())[0]
    return name_image

def bounding_box_detection(orig):
    min_size  = 200 #misura minima per le componenti connesse

    #convertiamo l'immagine in scala di grigi:
    gray = cv.cvtColor(np.array(orig), cv.COLOR_RGB2GRAY)

    #applichiamo il filtro gaussiano per rimuovere una parte del rumore:
    blur = cv.GaussianBlur(gray, (5, 5), 0)

    #Applichiamo la binarizzazione di otsu, con sfondo nero e oggetti bianchi
    _, binary_image = cv.threshold(blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    #applichiamo ora la  inversione dei colori in maniera tale da avere (come detto sopra) gli oggetti binachi su sfondo nero.
    binary_image = cv.bitwise_not(binary_image)

    #applichiamo un'altro filtro per il rumore: rimozione di oggetti piccoli:
    #per prima cosa prendiamo informazioni sulle varie componenti che sono nell'immagine:
    nb_components, output, stats, centroid = cv.connectedComponentsWithStats(binary_image, connectivity=8)
    size = stats[1:, -1]; nb_components = nb_components - 1

    #creiamo una immagine vuota per l'output:
    binary_image_cleaned = np.zeros((output.shape), np.uint8)

    #filtriamo le componenti piccole, in maniera tale da prendere da considerare solo quelle + grandi
    for i in range(0, nb_components):
        if size[i] >= min_size:
            binary_image_cleaned[output == i +1] = 255

    #Componenti connesse (celle) sull'immagine ripulita:
    lbl, numcc = ndimage.label(binary_image_cleaned)
    print("Numero di componenti connessi:", numcc)

    boxes = [] # Array che contiene le celle
    max_area = 0
    max_area_index = -1

    for i in range(1, numcc + 1):
        py, px = np.nonzero(lbl == i)
        if len(py) < min_size:
            continue

        xmin, xmax, ymin, ymax = px.min(), px.max(), py.min(), py.max()

        #calcolo delle bounding boxes (rettangolo mininmo che contiene le celle):
        boxes.append([
            [(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)],
            (np.mean(px), np.mean(py))
        ])
        #ora cosideriamo la componente connessa più grande (la griglia stessa)
        area = (xmax - xmin) * (ymax - ymin)
        if area > max_area:
            max_area = area
            max_area_index = i - 1
    return binary_image_cleaned.astype(np.uint8), boxes, max_area_index

def t_search(prob_array, labels):
    """funzione usata per trovare la posizione della testina T, passandogli la lista dei label
    e l'array delle probabilità

    Args:
        prob_array (array di float): array delle probabilità che i vari elementi della label siano la testina t
        labels (array int): array dei label per identificare quale è la testina.
    """
    prob_array = np.asarray(prob_array)

    #controllo della forma dell'array: l'array di input prob_array deve avere 3 dimensioni
    #e la forma del primo elemento deve essere (1, 4) perchè ogni elemento rappresenta la probabilità
    #per le 4 classi (B G Y T) per una songola cella

    if prob_array.ndim == 3 and prob_array[0].shape == (1, 4):
        index, max_prob = -1, 0

    #Individuazione della testina: scorre l'array di probabilità e trova la + alta per la classe T (che è il primo elemento dell'array [0]).
        for i in range(len(labels)):
            if prob_array[i][0][0] > max_prob:
                max_prob = prob_array[i][0][0]
                index = i
        if index != -1:
            print(f"La testina 'T' è all'indice {index} con probabilità {max_prob}")
            return index
        else :
            raise Exception("Nessun 'T' probabile è stato trovato nei dati")
    else:
        raise Exception("Array non di forma (X, 1, 4)")


def label_assigment(array, trueish_t):
        array = np.asarray(array)
        if array.ndim == 1:
            for a in range(len(array)):
                if a == trueish_t:
                    array[a] = 0 #assegnamo 0 alla posizione dell'etichetta della testina
            return array
        else:
            raise Exception("Non è un array 1-dimensinoale")

def cleandata(data, probabilities, labels):
    """Usa la fuinzione t_search per trovare la posizione più probabile della testina (trueish_t) in base alle probabilità
    e usa poi label_assigment per assegnare l'etichetta 0
    """
    return label_assigment(data, t_search(probabilities, labels))

def get_prob_array(model, data):
    """Funzione usata per ricevere l'array di probabilità partendo dal modello e dal data (il caso specifico)

    Args:
        model (keras.model): modello keras che sarà il core delle nostr preidzioni
        data (np.array): array che rappresenta l'immagine su cui lavorare.

    Returns:
        probabilities_array: array con le probabilità di ogni lettera sulle varie posizioni
        labels_array: array con le label più probabili per ogni casella
    """
    probabilities_Array = []
    labels_array = []
    for one_case in range(0, len(data)):
        prediction_single = model.predict(data[one_case:(one_case+1)])
        probabilities_Array.append(prediction_single)
        labels_array.append(prediction_single.argmax())
    return probabilities_Array, labels_array

def main():
    filename = import_image()
    orig = cv.imread(filename)
    orig = cv.cvtColor(orig, cv.COLOR_BGR2RGB)
    image_for_model, boxes, max_area_index = bounding_box_detection(orig)

    visual = image_for_model.copy()

    for b, centroid in boxes:
        #disegnamo il contorno delle bounding boxes (linea gialla)
        points = b.copy()
        points.append(points[0]) #Chiudi il poligono

        for i in range(len(points) - 1):
            pt1 = (int(points[i][0]), int(points[i][1]))
            pt2 = (int(points[i+1][0]), int(points[i+1][1]))
            cv.line(visual, pt1, pt2, (255, 255, 0), 2) #Giallo rgb

        #disegna il cerchio rosso sul Centroide
        cx, cy = centroid
        cv.circle(visual, (int(cx), int(cy)), 2, (255, 0, 0), -1)

    plt.figure(figsize=(10,10))
    plt.imshow(visual)
    plt.axis('off')
    plt.show()

    #Rimuoviamo la cella che corrisponde all'intera griglia:
    if max_area_index != -1 and len(boxes) > 1:
        boxes.pop(max_area_index)

    #Ordiniamo in base alla coordinata y del centro delle celle
    if boxes:
        boxes.sort(key=lambda x: x[1][1])

    table = [[]]
    row = 0
    y_row_prev = boxes[0][1][1] #coordinata y del centroide della prima cella

    table[row].append(boxes[0])

    for x in boxes[1:]:         #scorriamo le bounding box ordinate per coordinata y
        y_row_curr = x[1][1]    #coordinata y del centroide corrente

        if abs(y_row_prev -y_row_curr)<50: #confronto: se la differenza tra le y è <50, allora sono sulla stessa righa.
            table[row].append(x)
        else :                             #altrimenti su una riga nuova
            table.append([])
            row += 1
            table[row].append(x)

        y_row_prev = y_row_curr

    #Ordiniamo ogni riga per le coordinate x di ogni cella (da sx a dx)
    for r in table:
        r.sort(key=lambda x: x[1][0])


    #caldolo il numero di righe e colonne:
    h = len(table)
    w = max(len(row) for row in table)

    final_arr = []  #array che contiene le singole celle processate:


    #Iteriamo sulle celle individuate nella griglia e processiamo le singole immagini (sulle celle)
    for row in range(h):
        for col in range(len(table[row])):
            # Ottieni le cooridnate della cella corrente
            box = table[row][col]

            img = cv.imread(filename)

            #Corrdinate degli angoli opposti della bounding box per definire la regione rettangolare che racchiude la cella
            xmin = box[0][0][0]     # x dell' angolo in alto a sx
            ymin = box[0][0][1]     # y dell'angolo  in alto a sx
            xmax = box[0][2][0]     # x dell'angolo in basso a dx
            ymax = box[0][2][1]     #y dell'angolo in basso a dx

            cell_width = xmax -xmin
            cell_height = ymax - ymin

            # L'immagine ritagliata ha un margine del 7.5% perchè la casella risulta un po' restrittiva per le lettere
            cropped_image = img[int(ymin - cell_height * 0.075): int (ymax + cell_height*0.075),
                                int(xmin - cell_width * 0.075): int(xmax + cell_width * 0.075)]

            #convertiamo l'immagine in scala di grigi e applichiamo un filtro gaussiano
            gray = cv.cvtColor(cropped_image, cv.COLOR_BGR2GRAY)
            blur = cv.GaussianBlur(gray, (5, 5), 0)

            #applichiamo la sogli adattiva
            th_GAUSSIAN = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)

            #ridimensioniamo l'immagine
            resize = cv.resize(th_GAUSSIAN, (28, 28), interpolation = cv.INTER_CUBIC)

            #invertiamo i colori dell'immagine e la prepariamo per il modello:
            image_model_ready = cv.bitwise_not(resize)
            image_model_ready = image_model_ready.reshape(28,28)
            image_model_ready = image_model_ready.reshape(1, 784)
            image_model_ready = image_model_ready.astype("float32") / 255

            #in fine aggiungiamo l'immagine processata al final_Arr[]
            final_arr.append(image_model_ready)

    # Calcola il numero totale di immagini:
    num_image = len(final_arr)

    print(f"numero di celle : {num_image}")
    print(f"numero righe: {h}")
    print(f"numero colonne: {w}")

    #visualizzazione delle img processate:
    fig = plt.figure(figsize=(w*3, h*3)) #adatta le dimensioni della figura
    for i in range(num_image):
        row = i // w    # calcola la riga dell'immagine originale
        col = i % w     # calcola la colonna dell'immagine originale
        ax = fig.add_subplot(h, w, i+1) #posiziiona la sotto-figura nella posizione corretta:
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(False)
        ax.imshow(final_arr[i].reshape(28, 28), cmap = 'gray')
    plt.show()

    #calcolo delle probabilità e assegnazione delle etichette:
    probabilities, labels = get_prob_array(modello_filtrato, final_arr)
    cleanup = cleandata(data=labels, probabilities=probabilities, labels=labels)
    print("Etichette assegnate (B=1, Y=2, G=3, T=0):", cleanup)
    cleanup = [cleanup[a] for a in range(0, len(cleanup))]
    cleanup += [cleanup.index(0)]
    imm_testina = final_arr[cleanup[-1]]

    return cleanup, w, h, imm_testina

In [ ]:
main()

euristiche


In [ ]:
import aima
from aima import search

#Best First Search per poter usare A* come alg di ricerca
def best_first_search_graph_h(problem,  funzEur, no_memoize = False, t = -1) :
    init = Node(problem.initial)
    if problem.matriceGoal(init.state) :
        return init

    funzEur1 = funzEur
    funzEur2 = funzEur
    funzEur = memoize(funzEur, 'funzEur')
    #Frontiera di prio
    frontier = PriorityQueue('min', funzEur)
    frontier.append(init)

    explored = set()
    while frontier :
        node = frontier.pop()
        if (t == 0) and problem.cellaInit(node.state) :
            return node
        explored.add(tuple(node.state))
        for child in node.expand(problem) :
            if tuple(child.state) not in explored and child not in frontier :
                if (t == -1) and problem.matriceGoal(child.state) :
                  t = 0
                  funzEur = memoize(funzEur2, 'funzEur')
                  frontier = PriorityQueue('min', funzEur)
                  frontier.append(child)
                  break
                elif child in frontier:
                  incumbent = frontier.get_Item(child)
                  if funzEur(incumbent) > funzEur(child):
                    del frontier[incumbent]
                    frontier.append(child)

    return None

def A_Star(problem, h = None) :
    h = memoize(h or problem.h, 'h')
    eur =  best_first_search_graph_h(problem, lambda node : h(node), no_memoize = False)
    if eur is not None :
        return eur, eur.solution(), len(eur.solution())
    else :
        return None, None, 0

def h_maggiorColore(state):
    """Funzione che prende in input lo stato iniziale nel problema,
    e restituisce la label con maggiore concentrazione.

    Args:
        state (): Stato iniziale del problema

    Returns:
        label: label con maggiore presenza in questo stato.
    """
    contatore = {}
    for label in state[:-1]:
        if label not in contatore:
            contatore[label] = 0
        contatore[label] += 1

    maggiorColorePresente = max(contatore, key = contatore.get)
    return maggiorColorePresente

def h_color_difference(node, label_magg_freq):
    """Counter che indica sullo stato node quanti colori diversi ci sono,
    che non siano il più frequente

    Args:
        node (node): stato in cui siamo arrivati
        label_magg_freq (label): colore con maggiore frequenza

    Returns:
        int: numero di colori che non sono il più frequente.
    """
    label_maggiore_freq = label_magg_freq
    contatore_label_diff = 0 #contatore per quanti colori diversi ci sono
    for label in node.state[:-1]: #escludo la posizione della testina
        if label != label_maggiore_freq:
            contatore_label_diff += 1
    return contatore_label_diff

def h_manhattan(node, maggiorColorePresente, w) :
    labelPresentiNonMaxFreq = h_color_difference(node, maggiorColorePresente)
    #calcoliamo la distanza Manhattan dalla testian alla cella più vicina (con colore che non sia il maggioreColorePresente)
    posi_cursore = node.state[-1]
    min_distanza = float('inf')
    for i, label in enumerate(node.state[:-1]):
        if label != maggiorColorePresente:
            distanza = abs(posi_cursore // w - i // w) + abs(posi_cursore % w - i % w)
            min_distanza = min(min_distanza, distanza)
    return labelPresentiNonMaxFreq + min_distanza

visualizzazione della soluzione:

In [ ]:
!pip install ipywidgets
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib as plt
import numpy as np

#definiamo i colori presenti all'interno griglia: W (bianco), B (blue), Y (giallo), G (verde)

W = np.zeros((3, 3, 3, ), np.uint8)
Y = np.zeros((3, 3, 3, ), np.uint8)
B = np.zeros((3, 3, 3, ), np.uint8)
G = np.zeros((3, 3, 3, ), np.uint8)

# Inserimento dei colori nelle variabili:
W[:, :, :, ] = (255, 255, 255)      #colore bianco
Y[:, :, :, ] = (255, 255, 0)           #colore giallo
B[:, :, :, ] = (0, 0, 255)               #colore blu
G[:, :, :, ] = (0, 255, 0)              #colore verde

#lista dei colori
colori = [W, B, Y, G]

def visualizzatore_esecuzione(percorso_soluzione, movimento_soluzione, larghezza, altezza, testina):
    stato_soluzione = [i.state.copy() for i in percorso_soluzione]
    indice_stato_corrente = 0

    def a_pressione_bottone(b):
        nonlocal indice_stato_corrente
        if b.description == 'Successivo' and indice_stato_corrente < len(state_solution) - 1:
            indice_stato_corrente += 1
        elif b.description == 'Precedente' and indice_stato_corrente > 0:
            indice_stato_corrente -= 1

        clear_output(wait=True) #pulisce l'output precedente
        mostra_stato(indice_stato_corrente)

    bottone_prossimo = widgets.Button(description = "Successivo")
    bottone_precedente = widgets.Button(description = "Precedente")
    bottone_prossimo.on_click(a_pressione_bottone)
    bottone_precedente.on_click(a_pressione_bottone)

    def mostra_stato(indice): #amici di monster allergy

        stato = stato_soluzione[indice]
        T_indice = stato[:-1]

        fig = plt.figure(figsize = (larghezza * 2, altezza * 2))
        for i in range(len(state)):
            plt.subplot(h, w, i+1)
            plt.xticks([])
            plt.yticks([])
            if i == T_indice:
                #mostra la testina nella cella corrente
                plt.imshow(testina.reshape(28, 28), cmap='gray')
            else:
                #mostra il colore della cella
                plt.imshow(colori[stato[i]])

        display(bottone_precedente)
        display(bottone_prossimo)

        if indice == 0:
            plt.suptitle(f"STATO INIZIALE, PROSSIMA AZIONE: {movimento_soluzione[indice]}")
        elif indice <= len(movimento_soluzione) -1:
            plt.suptitle(f"AZIONE APPLICATA: {movimento_soluzione - 1} | PROSSIMA AZIONE: {movimento_soluzione[indice]}")
        else:
            plt.suptitle(f"STATO FINALE, AZIONE APPLICATA: {movimento_soluzione[indice -1]}")
        plt.show()

    mostra_stato(indice_stato_corrente)

demo

In [ ]:
from tensorflow import keras
import time

#definizione di parte[]:
#parte[0] è lo stato iniziale del problema
#parte[1]: è la larghezza della griglia
#parte[2] è l'altezza della griglia.
result = main()
parte = result[:3]
testina = result[3]
problema = ProblemaColori(parte[0], parte[1], parte[2])
label_magg_usate = h_maggiorColore(parte[0])

avvio_tempo_del_mondo = time.time()
soluzione, azione, lunghezza = A_Star(problema, lambda node : h_manhattan(node, label_magg_usate, parte[1]))

fine_tempo_del_mondo = time.time()

if soluzione is not None:
    visualizzatore_esecuzione(soluzione.path(), soluzione.solution(), parte[1], parte[2], testina)

    print('Azione = ', azione)
    print('Costo = ', soluzione.path_cost)
    print('Lunghezza = ', lunghezza)
    print('Tempo = ', fine_tempo_del_mondo - avvio_tempo_del_mondo)
else:
    print("DATI INSUFFICIENTI PER RISPOSTA SIGNIFICATIVA.")